# Approaches

* `tflite_convert`: Starting from `TensorFlow 1.9`, the command-line tool tflite_convert is installed as part of the Python package. All of the examples below use tflite_convert for simplicity.
    * Example: `tflite_convert --output_file=...`
    
* `bazel`: In order to run the latest version of the TensorFlow Lite Converter either install the nightly build using pip or clone the TensorFlow repository and use bazel.
    * Example: `bazel run //tensorflow/contrib/lite/python:tflite_convert -- --output_file=...`
    
* `Python API`: Tensroflow version < 1.9

# Examples

## Convert a TensorFlow GraphDef

Make sure the frozen model file `.pb` is exported from [freeze_graph.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py).

```sh
# you can download the example model 
curl https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_0.50_128_frozen.tgz \
  | tar xzv -C /tmp 
  
# convert the model to tflite
tflite_convert \
    --output_file=/tmp/foo.tflite \
    --graph_def_file=/tmp/mobilenet_v1_0.50_128/frozen_graph.pb \
    --input_arrays=input \
    --output_arrays=MobilenetV1/Predictions/Reshape_1
```

## Convert from SavedModel

`SaveModel` is the checkpoint-based way to conserve the model. Usually it is the folder conversing a checkpoint, and lots of `.ckpt` or `ckpt-{epoch}` style filename of files.

The values for `--input_arrays` and `--output_arrays` are an aggregated, alphabetized list of the inputs and outputs in the SignatureDefs within the `MetaGraphDef` specified by `--saved_model_tag_set`.

```sh
tflite_convert \
    --output_file=/tmp/foo.tflite \
    --saved_model_dir=/tmp/saved_model
```

## Conver from `tf.keras` model

```sh
tflite_convert \
    --output_file=/tmp/foo.tflite \
    --keras_model_file=/tmp/keras_model.h5
```

# Quantization

The TensorFlow Lite Converter is compatible with fixed point quantization models. (refer to webpage: https://www.tensorflow.org/performance/quantization).

## UINT8

```sh
tflite_convert \
    --output_file=/tmp/foo.tflite \
    --graph_def_file=/tmp/some_quantized_graph.pb \
    --inference_type=QUANTIZED_UINT8 \
    --input_arrays=input \
    --output_arrays=MobilenetV1/Predictions/Reshape_1 \
    --mean_values=128 \
    --std_dev_values=127
```

## Dummy-quantization

"Dummy-quantization" will produce lower accuracy but will emulate the performance of a correctly quantized model.

```sh
curl https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_0.50_128_frozen.tgz \
    | tar xzv -C /tmp
tflite_convert \
    --output_file=/tmp/foo.cc \
    --graph_def_file=/tmp/mobilenet_v1_0.50_128/frozen_graph.pb \
    --inference_type=QUANTIZED_UINT8 \
    --input_arrays=input \
    --output_arrays=MobilenetV1/Predictions/Reshape_1 \
    --default_ranges_min=0 \
    --default_ranges_max=6 \
    --mean_values=128 \
    --std_dev_values=127
```

# Multiple Input/Output Arrays

## Multiple Input Arrays

Note the `input_shape` is `colon-separated` list, the `input_array` is `comma-separated` list.

```sh
curl https://storage.googleapis.com/download.tensorflow.org/models/inception_v1_2016_08_28_frozen.pb.tar.gz \
  | tar xzv -C /tmp
tflite_convert \
    --graph_def_file=/tmp/inception_v1_2016_08_28_frozen.pb \
    --output_file=/tmp/foo.tflite \
    --input_shapes=1,28,28,96:1,28,28,16:1,28,28,192:1,28,28,64 \
    --input_arrays=InceptionV1/InceptionV1/Mixed_3b/Branch_1/Conv2d_0a_1x1/Relu,InceptionV1/InceptionV1/Mixed_3b/Branch_2/Conv2d_0a_1x1/Relu,InceptionV1/InceptionV1/Mixed_3b/Branch_3/MaxPool_0a_3x3/MaxPool,InceptionV1/InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/Relu \
    --output_arrays=InceptionV1/Logits/Predictions/Reshape_1
```

## Multiple Output Arrays

```sh
curl https://storage.googleapis.com/download.tensorflow.org/models/inception_v1_2016_08_28_frozen.pb.tar.gz \
  | tar xzv -C /tmp
tflite_convert \
  --graph_def_file=/tmp/inception_v1_2016_08_28_frozen.pb \
  --output_file=/tmp/foo.tflite \
  --input_arrays=input \
  --output_arrays=InceptionV1/InceptionV1/Mixed_3b/Branch_1/Conv2d_0a_1x1/Relu,InceptionV1/InceptionV1/Mixed_3b/Branch_2/Conv2d_0a_1x1/Relu
```

## Specifying subgraphs

Any array in the input file can be specified as an input or output array in order to extract subgraphs out of an input graph file. The TensorFlow Lite Converter discards the parts of the graph outside of the specific subgraph.

```sh
curl https://storage.googleapis.com/download.tensorflow.org/models/inception_v1_2016_08_28_frozen.pb.tar.gz \
  | tar xzv -C /tmp
tflite_convert \
  --graph_def_file=/tmp/inception_v1_2016_08_28_frozen.pb \
  --output_file=/tmp/foo.pb \
  --input_shapes=1,28,28,96:1,28,28,16:1,28,28,192:1,28,28,64 \
  --input_arrays=InceptionV1/InceptionV1/Mixed_3b/Branch_1/Conv2d_0a_1x1/Relu,InceptionV1/InceptionV1/Mixed_3b/Branch_2/Conv2d_0a_1x1/Relu,InceptionV1/InceptionV1/Mixed_3b/Branch_3/MaxPool_0a_3x3/MaxPool,InceptionV1/InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/Relu \
  --output_arrays=InceptionV1/InceptionV1/Mixed_3b/concat_v2
```

# Logging

You can convert or export the graph visualization and please refer to official document link https://www.tensorflow.org/lite/convert/cmdline_examples.